In [2]:
import glob
import pandas as pd
import numpy as np

In [6]:
fileList = glob.glob("book_train.parquet/*")

In [19]:
df_book_data = pd.read_parquet(fileList[0])
df_book_data.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,1.001186,1.001581,1.001044,1.001750,100,200,63,100
1,5,2,1.001186,1.001581,1.001016,1.001750,100,100,63,100
2,5,4,1.001186,1.001750,1.001016,1.002032,176,100,63,2
3,5,5,1.001355,1.001750,1.001186,1.001891,100,100,76,63
4,5,6,1.001355,1.001750,1.001186,1.002032,200,100,76,2


In [20]:
df_book_data['wap'] = (df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1']) / (df_book_data['bid_size1']+ df_book_data['ask_size1'])
df_book_data.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,wap
0,5,0,1.001186,1.001581,1.001044,1.001750,100,200,63,100,1.001317
1,5,2,1.001186,1.001581,1.001016,1.001750,100,100,63,100,1.001383
2,5,4,1.001186,1.001750,1.001016,1.002032,176,100,63,2,1.001546
3,5,5,1.001355,1.001750,1.001186,1.001891,100,100,76,63,1.001552
4,5,6,1.001355,1.001750,1.001186,1.002032,200,100,76,2,1.001618


In [21]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 
df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
df_book_data.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,wap,log_return
0,5,0,1.001186,1.001581,1.001044,1.001750,100,200,63,100,1.001317,NaN
1,5,2,1.001186,1.001581,1.001016,1.001750,100,100,63,100,1.001383,0.000066
2,5,4,1.001186,1.001750,1.001016,1.002032,176,100,63,2,1.001546,0.000162
3,5,5,1.001355,1.001750,1.001186,1.001891,100,100,76,63,1.001552,0.000007
4,5,6,1.001355,1.001750,1.001186,1.002032,200,100,76,2,1.001618,0.000066


In [44]:
slice = df_book_data[(df_book_data['time_id'] == 109) & (df_book_data['seconds_in_bucket'] > 10)]
slice.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,wap,log_return
2953,109,11,0.999156,0.999522,0.998903,0.999634,125,56,100,5,0.999409,1.377777e-04
2954,109,12,0.999240,0.999634,0.999184,0.999916,67,5,100,64,0.999607,1.984620e-04
2955,109,13,0.999269,0.999634,0.999156,0.999916,125,5,100,64,0.999620,1.335662e-05
2956,109,14,0.999269,0.999634,0.999240,0.999916,125,5,67,64,0.999620,0.000000e+00
2957,109,15,0.999044,0.999634,0.998875,0.999916,200,5,67,64,0.999620,-3.577443e-07


In [ ]:
df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()
df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
stock_id = file_path.split('=')[1]
df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')